In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import Variable

### Defining XorNet for learning XOR operaton

In [ ]:
class XorNet(nn.Module):
    def __init__(self):
        super(XorNet, self).__init__()
        self.fc1 = nn.Linear(2, 10)     # 2 input neurons, 10 hidden neurons
        self.fc2 = nn.Linear(10, 1)     # 1 output neuron
    def forward(self, x):
        x = F.relu(self.fc1(x))         # relu is used as non linear activation function 
        x = self.fc2(x)
        return x

In [ ]:
class Model(object):
    def __init__(self, net, learning_rate = 0.01, epoch=100):
        self.net = net 
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.criterion = nn.MSELoss()
        self.optimizer = optim.SGD(self.net.parameters(), self.learning_rate)
        self.net.zero_grad()
    def generate_data(self):
        x = (torch.rand(10, 2)>0.5)       #generate random data; batch size 10
        y = np.logical_xor(x[:,0],x[:,1])
        y.view(-1,1)
        x = Variable(x.type(torch.FloatTensor))
        y = Variable(y.type(torch.FloatTensor))
        return x, y
    def decision_boundary(self, itr=-1):
        plt.figure(figsize=(5,5))
        x = np.arange(0.0, 1.001, 0.01)
        y = np.arange(0.0, 1.001, 0.01)
        data = torch.zeros(len(x) * len(y), 2)
        for i in range(len(x)):
            for j in range(len(y)):
                data[len(x) * i + j][0] = x[i]
                data[len(x) * i + j][1] = x[j]
        data = Variable(data)
        output = self.net(data)
        z = output.view(len(x),len(y)).data.numpy()
        plt.contourf(x,y,z)
        plt.savefig('xor_%d.png' %itr)
        plt.show()
        
    def training(self):
        loss_history = []
        for i in range(self.epoch):
            x, y = self.generate_data()
            self.optimizer.zero_grad()   # zero the gradient buffers
            output = self.net(x)
            loss = self.criterion(output, y)
            loss_history.append(loss)
            loss.backward()
            self.optimizer.step()
            self.net.eval()
            self.decision_boundary(i)
            self.net.train()
            print("Number of training:" +str(i))
        return loss_history
    def test(self):
        x, y = self.generate_data()
        return x, self.net(x), y
   # def test1(self, x):
   #     return self.net(x)

### Training XOR Net 

In [ ]:
net=XorNet()
model = Model(net, 0.01, 500)
loss_history = model.training()
plt.scatter(range(len(loss_history)), loss_history)
plt.show()

### Testing XOR Net

In [ ]:
data, output, target = model.test()
print("Randomly Generated Test Data:")
print (data.data[:,:])
print("Output Comparison with the target output:")
plt.bar(np.array(range(len(output)))-0.14, output, label='output', color='r', width= 0.3)
plt.bar(np.array(range(len(output)))+0.14, target, label='target', color='g', width= 0.3)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
x = np.arange(0.0, 1.001, 0.01)
y = np.arange(0.0, 1.001, 0.01)

data = torch.zeros(len(x) * len(y), 2)
for i in range(len(x)):
    for j in range(len(y)):
        data[len(x) * i + j][0] = x[i]
        data[len(x) * i + j][1] = x[j]
data = Variable(data)
output = model.test1(data)
z = output.view(len(x),len(y)).data.numpy()
plt.contourf(x,y,z)
plt.savefig('xor_25.png')
plt.show()
